In [134]:
#set up smart sensing of os
if(Sys.info()[1]=='Windows'){
  j = "J:/"
} else{
  j = '/home/j/'
}
options(scipen=999)

In [135]:
#collect arguments
argue = commandArgs(trailingOnly = T)
task_id <- 8 #as.numeric(Sys.getenv("SGE_TASK_ID"))
slots= 29 #as.numeric(argue[1])
year = 1990 #as.numeric(argue[2])
temperature_product = 'era_mean' #as.character(argue[3]) #'era_mean' #
admin_level = 'admin0' #as.character(argue[4])#'admin0' #'isestimate'
the_cause = 'cvd_ihd' #as.character(argue[5]) #'diabetes'#make sure to do the memory math, in qsub, use 30+ slots
paf_version = 2 #as.character(argue[6]) #'test'
tmrel_version = 1 # as.character(argue[7]) #'test'
risk_version = 'test' #as.character(argue[8]) #test'
convert_kelvin = T #as.logical(argue[9]) #T

In [136]:
#set conditional variables
draw = task_id -1
data.dir = paste0('/share/geospatial/temperature/estimates/', temperature_product,'/')
cores_to_use = ifelse(grepl('Intel', system("cat /proc/cpuinfo | grep \'name\'| uniq", inter = T)), floor(slots * .86), floor(slots*.64))
pop.dir = paste0(j,'WORK/01_covariates/02_inputs/population_counts/outputs/full_ts_1980_2015/')
data_product = substr(temperature_product, 1,regexpr('\\_', temperature_product)-1)


In [137]:
#check to make sure things passed properly
print(commandArgs())
print(paste(task_id, draw, slots, year, temperature_product,admin_level, the_cause, paf_version, tmrel_version,risk_version,convert_kelvin))

[1] "/usr/local/codem/public_use_anaconda/lib/R/bin/exec/R"                                            
[2] "--slave"                                                                                          
[3] "-e"                                                                                               
[4] "IRkernel::main()"                                                                                 
[5] "--args"                                                                                           
[6] "/snfs2/HOME/dccasey/.local/share/jupyter/runtime/kernel-4c023e0a-3006-4780-8456-0103fb4f98da.json"
[1] "8 7 29 1990 era_mean admin0 cvd_ihd 2 1 test TRUE"


In [138]:
#load libraries
pack_lib = '/home/j/temp/dccasey/temperature/packages/'
.libPaths(pack_lib)
library('parallel')
for(ppp in c('data.table','raster','ncdf4', 'sp', 'rgdal', 'pryr','profvis')){
  suppressMessages(library(ppp, lib.loc = pack_lib, character.only =T))
}

source(paste0('/home/j/temp/central_comp/libraries/current/r/get_location_metadata.R'))


In [139]:
#set raster options
num_cells = round(((slots*2)-20)/7) * 1e9 #leave some overhead for other memory
rasterOptions(maxmemory = num_cells) #1e9 is like 7 gigs I think

In [140]:
#find and load the proper dataset
expo = brick(paste0(data.dir,list.files(path = data.dir, pattern = as.character(year))))
#convert to degrees C
if(convert_kelvin) expo = expo -273.15

#decide if things need rotation. If xmax is >183 (180, but with a lil offset), rotate things
rotate_me = ifelse(extent(expo)[2] >181, T, F)
if(rotate_me) expo = rotate(expo)

In [141]:
#find and load relevant population grid
pop = raster(paste0(pop.dir, 'glp',year,'ag.tif'))

In [142]:
#crop expo to the pop grid
expo = crop(expo, pop)

In [143]:
#find and load relevant rasterized locations
locs = raster(paste0(j,'/temp/dccasey/temperature/data/rasterized_shapefiles/',data_product, admin_level,'.tif')) #should be more flexible to get era interim working
locs = crop(locs, pop)

In [144]:
#aggregate population to the exposure cell size
pop_fact = round(dim(pop)[1:2] / dim(expo)[1:2])
pop = aggregate(pop, pop_fact)
pop = resample(pop, expo)

In [145]:
#load tmrel brick
load(paste0('/share/geospatial/temperature/estimates/tmrel/tmrel_', tmrel_version,'.Rdata'))
#for now, tmrel is a single draw/value tmrel = tmrel[[draw + 1]] #tmrel is index 1-1000 I think

In [146]:
#resample TMREL is the bricks don't match
tmrel = crop(tmrel, expo)
tmrel = resample(tmrel,expo)

In [147]:
#load risk brick
load(paste0('/share/geospatial/temperature/estimates/risk/temperature_risks_', risk_version, '.Rdata'))

In [148]:
#keep relevant draw
risk = risk_grid[,.(acause, age_group_id, sex_id, element, measure, risk = get(paste0('pc_',draw)))]
rm(risk_grid)

In [149]:
#convert relevant objects to data tables
expo = setDT(as.data.frame(expo, xy = T))
pop = setDT(as.data.frame(pop, xy = T))
tmrel = setDT(as.data.frame(tmrel, xy= T))
locs = setDT(as.data.frame(locs, xy = T))

In [150]:
#set names
names(expo)[3:length(expo)] = paste0('day_',(3:length(expo))-2)
names(pop) = c('x','y','pop')
names(tmrel) = c('x','y','tmrel')
names(locs) = c('x','y','location_id')

In [151]:
#cbind everything together
temp_dat = do.call(cbind, list(expo, pop = pop[,pop], tmrel = tmrel[,tmrel], location_id = locs[,location_id]))

In [152]:
#clean space
lapply(list(expo, pop, tmrel, locs), function(x) rm(x))

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

In [153]:
#subset by where there is a location id
temp_dat = temp_dat[!is.na(location_id),]
blerg = copy(temp_dat)

In [154]:
#by risk, convert to degrees from threshold, convert to RR, find numerator and denominator by year, calculate paf

In [155]:
#convert data table with temperature into RR
#td: temperature dataset/datatable
#elem: heat or cold
#risk_val: what is the % change in risk with one degree from the threshold
#adj: used to convert from percentage point space to proportion/percent
calc_deg_thresh = function(td, elem, tmrel_col){
    #prevent scoping issues
    td = copy(td)
    
    #convert into degrees from threshold
    day_cols = grep('day', names(td), value = T)
    td = td[, (day_cols) := lapply(day_cols, function(x) get(x) - get(tmrel_col))]
    td = td[,row_id:= 1:nrow(td)]
    #adjust by hot or cold
    for(ddd in day_cols){
        if(elem == 'heat'){
            zero_rows = td[get(ddd)<0, row_id]
        } else{
            zero_rows = td[get(ddd)>0, row_id]
        }
       set(td, zero_rows, j = ddd, value = 0)
    }
        
    #make absolute value and convert to risk
    #td = td[, day_cols := lapply(day_cols, function(x) 1+(get(x) * rr))]
    
    #calculate the absolute number of degrees over the threshold in that year
    dd = td[, abs(rowSums(.SD)), .SDcols=day_cols]
    
    return(dd)
    
}

In [156]:
#find the degrees from threshold for hot and cold
#only works if TMREL has no uncertainty. If TMREL has uncertainty, change the lapply to do once per draw (use mclappy?)
#be careful of memory
thresholds = lapply(c('heat','cold'), function(el) calc_deg_thresh(temp_dat,el, 'tmrel'))

In [157]:
#drop the unneed days columns
day_cols = grep('day', names(temp_dat),value = T)
temp_dat = temp_dat[,names(temp_dat)[!grepl('day', names(temp_dat))],with=F]
temp_dat = temp_dat[,paste0('deg_days_',c('heat','cold')) := thresholds]

In [158]:
mem_used()

1.51 GB

In [159]:
hold = copy(temp_dat)

In [203]:
#for each risk-age-element-sex-cause, calculate pixel level risk
#(risk * degrees over threshold) +1
temp_dat = temp_dat[, paste0('risk_',1:nrow(risk)) := lapply(1:nrow(risk), function(x){
    (1+((risk[x,risk] * .01) *get(paste0('deg_days_',risk[x,element]))/365))
})]

In [205]:
head(temp_dat)

x,y,pop,tmrel,location_id,deg_days_heat,deg_days_cold,risk_1,risk_2,risk_3,⋯,risk_406,risk_407,risk_408,risk_409,risk_410,risk_411,risk_412,risk_413,risk_414,risk_1_val
-36.5,83.5,0,-21.98125,349,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
-36.0,83.5,0,-22.38333,349,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
-35.5,83.5,0,-22.45833,349,2774.936,2423.491,1.163562,1.015996,0.8688977,⋯,1.431413,1.403797,0.7768923,1.720444,1.926475,1.565357,1.689055,1.520352,1.576968,1.163562
-35.0,83.5,0,-21.87083,349,2667.814,2565.444,1.157248,1.016933,0.8739587,⋯,1.456682,1.427449,0.7638240,1.762643,1.980742,1.598472,1.729416,1.550831,1.610763,1.157248
-34.5,83.5,0,-21.45208,349,2602.497,2622.679,1.153398,1.017311,0.8770446,⋯,1.466871,1.436986,0.7585549,1.779657,2.002622,1.611824,1.745689,1.563120,1.624389,1.153398
-33.5,83.5,0,-21.72708,349,2652.461,2511.646,1.156343,1.016578,0.8746840,⋯,1.447106,1.418485,0.7687767,1.746650,1.960175,1.585922,1.714119,1.539279,1.597955,1.156343


In [221]:
pafs = temp_dat[,lapply(1:nrow(risk), function(x){
    sum((get(paste0('risk_',x))-1)*pop ,na.rm=T)/sum(get(paste0('risk_',x)) *pop,na.rm=T)
}),by = 'location_id']

In [225]:
mem_used()

1.6 GB

In [226]:
head(pafs)

location_id,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V405,V406,V407,V408,V409,V410,V411,V412,V413,V414
349,0.07419136,0.012130079,-0.06864257,0.03522860,0.06201076,0.019844293,0.02839627,0.015544079,0.07071981,⋯,0.4983233,0.2487791,0.23662247,-0.20665788,0.3560998,0.4156119,0.3026431,0.3459524,0.2854278,0.3069507
101,0.10819023,0.008203820,-0.10771407,0.02400752,0.09097627,0.013455109,0.04236955,0.010524550,0.10330532,⋯,0.4008892,0.1823973,0.17273853,-0.13041772,0.2714284,0.3239083,0.2262168,0.2627094,0.2120270,0.2297951
62,0.10247617,0.008608945,-0.10073710,0.02517326,0.08608360,0.014115865,0.03997598,0.011043002,0.09782096,⋯,0.4126185,0.1897587,0.17979584,-0.13780924,0.2811474,0.3346434,0.2348388,0.2722329,0.2202621,0.2385113
102,0.08703988,0.006643141,-0.08274084,0.01949948,0.07291597,0.010906420,0.03360181,0.008526175,0.08302103,⋯,0.3510678,0.1528039,0.14443542,-0.10287201,0.2314797,0.2791955,0.1911762,0.2236500,0.1786769,0.1943394
90,0.07863525,0.004270399,-0.07343268,0.01259303,0.06577689,0.007021719,0.03018658,0.005484580,0.07497259,⋯,0.2575747,0.1036752,0.09768704,-0.06362371,0.1618892,0.1989740,0.1316271,0.1559358,0.1224318,0.1339682
79,0.08405412,0.006146664,-0.07939606,0.01805965,0.07037736,0.010094565,0.03238413,0.007890088,0.08016103,⋯,0.3334718,0.1429561,0.13504095,-0.09440600,0.2178653,0.2637379,0.1793790,0.2103701,0.1674913,0.1823910
